# CELLEX

In [1]:
import pandas as pd
import cellex

In [2]:
data_dir = 'data/CELLEX'
file = data_dir+'/DroNc/GTEx_droncseq_hip_pcf/GTEx_droncseq_hip_pcf.umi_counts.txt'

In [3]:
%%time
data = pd.read_csv(file, 
                   index_col=0,
                   sep='\t')

CPU times: user 1min 42s, sys: 4.29 s, total: 1min 46s
Wall time: 1min 46s


In [4]:
data

,hHP1_AACACTATCTAC,hHP1_CTACGCATCCAT,hHP1_TCGGTACTAATA,hHP1_CCCGCACGCTAT,hHP1_TCATTTTGTCAT,hHP1_ACGAGGTCTATG,hHP1_AGTCATGAGGTT,hHP1_GTTAGTATACCA,hHP1_GCATTCAGTAAG,hHP1_AGACCGCGACTA,...,PFC-CD_CTCCCGTAGAAC,PFC-CD_ATATCGCCCCAT,PFC-CD_GGTGATAGACCC,PFC-CD_TGTGCGGCTCGN,PFC-CD_CAACCAATTTCG,PFC-CD_TTGCCTGGCGGG,PFC-CD_CACGCTCCCCTA,PFC-CD_GCTCTACAACCG,PFC-CD_CTCCATTCATGC,PFC-CD_CGTCATTAGCAG
A1BG,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZYG11B,0,1,2,0,3,1,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ZYX,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZZEF1,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [5]:
metafile = data_dir+'/DroNc/GTEx_droncseq_hip_pcf/GTEx_droncseq_hip_pcf.clusters.txt'
metadata = pd.read_csv(metafile, sep='\t', header=None, names=['cell','cluster'])
display(metadata.cluster.unique()) # display clusters

array([ 4,  3,  2, 19,  1, 18,  8,  5, 11,  7, 15,  6, 16, 13,  9, 12, 10,
       17, 14])

In [6]:
# change numerical clusters to more meaningful cluster names
clusters ={1:"exPFC1", 2:"exPFC2", 3:"exCA1", 4:"exCA3", 5:"GABA1", 
           6:"GABA2", 7:"exDG", 8:"ASC1", 9:"ASC2", 10:"ODC1", 11:"ODC2", 
           12:"OPC", 13:"MG", 14:"NSC", 16:"END"}
metadata['cluster'] = metadata['cluster'].map(clusters)
metadata.set_index('cell')
display(metadata)

,cell,cluster
0,hHP1_AACACTATCTAC,exCA3
1,hHP1_CTACGCATCCAT,exCA1
2,hHP1_TCGGTACTAATA,exCA1
3,hHP1_CCCGCACGCTAT,exCA1
4,hHP1_TCATTTTGTCAT,exCA1
...,...,...
14958,PFC-CD_TTGCCTGGCGGG,NaN
14959,PFC-CD_CACGCTCCCCTA,exPFC1
14960,PFC-CD_GCTCTACAACCG,END
14961,PFC-CD_CTCCATTCATGC,NaN


In [7]:
# remove cells from a nan clusters
nan_cells = metadata[metadata['cluster'].isnull()].cell.tolist()
len(nan_cells) # number of nan cells

826

In [8]:
data_dropped = data.drop(nan_cells, axis=1)
print(data.shape, data_dropped.shape)

(32111, 14963) (32111, 14137)


In [9]:
metadata_dropped = metadata.dropna()
metadata_dropped.set_index('cell', inplace=True)
print(metadata_dropped.shape)

(14137, 1)


In [10]:
%%time 
eso = cellex.ESObject(data=data_dropped,
                      annotation=metadata_dropped,
                      verbose=True)

Preprocessing - checking input ... input parsed in 0 min 0 sec
Preprocessing - running remove_non_expressed ... excluded 276 / 32111 genes in 0 min 24 sec
Preprocessing - normalizing data ... data normalized in 0 min 44 sec
Preprocessing - running ANOVA ... excluded 18019 / 31835 genes in 0 min 18 sec
CPU times: user 26.7 s, sys: 1min, total: 1min 27s
Wall time: 1min 28s


In [11]:
%%time 
eso.compute(verbose=True)

Computing DET ... 
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 3 sec
Computing EP ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 0 sec
Computing GES ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 1 sec
Computing NSI ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 0 sec
Computing ESmu ...
    finished in 0 min 0 sec
Computing ESsd ...
    finished in 0 min 0 sec
Computed ['det.esw', 'det.esw_null', 'det.pvals', 'det.esw_s', 'ep.esw', 'ep.esw_null', 'ep.pvals', 'ep.esw_s', 'ges.esw', 'ges.esw_null', 'ges.pvals', 'ges.esw_s', 'nsi.esw', 'nsi.esw_null', 'nsi.pvals', 'nsi.esw_s', 'esmu', 'essd'].
CPU times: user 5.08 s, sys: 1.27 s, total: 6.35 s
Wall time: 6.34 s


In [12]:
# cellex.utils.mapping.human_symbol_to_human_ens(eso.results["esmu"], drop_unmapped=True, verbose=True) 
def mapping_symbols_to_ens_37(df):
    shape_before = df.shape[0]
    ENSG_HGNC_df = pd.read_csv(data_dir+'/Homo_sapiens.GRCh37.ENS.HGNC.txt')
    ENSG_HGNC_df = ENSG_HGNC_df[ENSG_HGNC_df['Ensembl Gene ID'].str.startswith('ENSG')].dropna()
    ENSG_HGNC_df.set_index('HGNC symbol', inplace=True)
    ENSG_HGNC_dict = ENSG_HGNC_df.to_dict()['Ensembl Gene ID']
    
    df.index = df.index.map(ENSG_HGNC_dict)
    df = df[df.index.notnull()]
    print(f" {100*(shape_before-df.shape[0])/shape_before:.4} pct of genes are unmapped ...")
    print(f"Removed {(shape_before-df.shape[0])} unmapped genes ...")
    return df

df = mapping_symbols_to_ens_37(eso.results["esmu"])

 10.57 pct of genes are unmapped ...
Removed 1460 unmapped genes ...


The ```cellex.utils.mapping.human_symbol_to_human_ens``` method converts symbols to ENS gene id's using GRCh38.
It can still be used but 1794 genes (12.98%) cannot be mapped and are thus removed.


This datsets however used GRCh37 so instead I mapped using a different file acquired from BioMart using the ```mapping_symbols_to_ens_37``` function. With this, 1460 (10.57%) of the genes cannot be mapped.

In [13]:
df

,ASC1,ASC2,END,GABA1,GABA2,MG,NSC,ODC1,ODC2,OPC,exCA1,exCA3,exDG,exPFC1,exPFC2
gene,,,,,,,,,,,,,,,
ENSG00000268895,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.050963,0.0,0.015238,0.125513,0.000000,0.354698,0.000000
ENSG00000175899,0.000000,0.000000,0.798519,0.000000,0.000000,0.57891,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000166535,0.884156,0.722656,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000128274,0.012620,0.000000,0.889029,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000094914,0.011776,0.000000,0.000000,0.000000,0.000000,0.00000,0.269709,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.047000,0.334751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000153975,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.354246,0.000000,0.000000,0.000000,0.247950
ENSG00000122952,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.792194,0.677215,0.000000,0.118185,0.000000
ENSG00000162378,0.000000,0.000000,0.000000,0.332288,0.398926,0.00000,0.000000,0.0,0.000000,0.0,0.105593,0.000000,0.211740,0.283113,0.285233


In [14]:
# df.to_csv("GTEx_droncseq_hip_pcf.esmu.csv.gz")